# Gradient descent and stochastic gradient descent 

## Why gradient matters

In previous chapters on training deep learning models, you may have encountered the routines of updating parameter values based on computing gradients of objective functions. We did so in the hope that the values of objective functions may be iteratively reduced. Indeed, such routines are crucial for optimization algorithms used in the model training. You may wonder, why can we rely on such gradients to update model parameters for optimization? 

### Gradient descent in one-dimensional domain

To answer this question, we need to formalize the problem in mathematics. We have tried to keep the mathematical content to the minumum necessary to get a proper understanding. The emphasis in this chapter is on conveying the underlying concepts rather than on the mathematical rigour.


Let us consider a simple scenario with the objective function $f: \mathbb{R} \rightarrow \mathbb{R}$. Note that the domain of $f$ in one-dimensional. According to its Taylor series expansion as shown in the [introduction chapter](./P13-C01-intro.ipynb), we have

$$f(x + \epsilon) \approx f(x) + f'(x) \epsilon.$$

Substituting $\epsilon$ with $-\eta f'(x)$ where $\eta$ is a constant, we have

$$
\begin{align}
f(x - \eta f'(x)) &\approx f(x) -  \eta f'(x)^2. \\
\end{align}
$$

If $\eta$ is set as a small positive value and $f'(x) \neq 0$, we obtain

$$f(x - \eta f'(x)) < f(x).$$

In other words, updating $x$ as 

$$x := x - \eta f'(x)$$ 

may reduce the value of $f(x)$. Since the derivative $f'(x)$ is a special case of gradient in one-dimensional domain, the above update of $x$ is gradient descent in one-dimensional domain.

The positive scalar $\eta$ is called the learning rate or step size. Note that a larger learning rate increases the chance of overshooting the global minimum and oscillating. However, if the learning rate is too small, the convergence can be very low. In practice, a proper learning rate is usually selected with experiments.

### Gradient descent  in multi-dimensional domain$\newcommand{\xb}{\mathbf{x}} \newcommand{\ub}{\mathbf{u}}$

We go on to illustrate the idea of gradient descent in multi-dimensional domain. Consider the objective function $f: \mathbb{R}^d \rightarrow \mathbb{R}$ that takes any vector $\xb = [x_1, x_2, \ldots, x_d]^\top$ as the input. The gradient of $f(\xb)$ with respect to $\xb$ is defined by the vector of partial derivatives as 

$$\nabla_\xb f(\xb) = \bigg[\frac{\partial f(\xb)}{\partial x_1}, \frac{\partial f(\xb)}{\partial x_2}, \ldots, \frac{\partial f(\xb)}{\partial x_d}\bigg]^\top.$$

We use the notation $\nabla f(\xb)$ and $\nabla_\xb f(\xb)$ exchangeably when there is no ambiguity.
In plain English, each element of the gradient $\partial f(\xb)/\partial x_i$ indicates the rate of change for $f$ at the point $\xb$ only in correspondance to the increase of $x_i$. To measure the rate of change of $f$ in any direction that is represented by a unit vector $\ub$, in multivariate calculus we define the directional derivative of $f$ at $\xb$ in the direction of $\ub$ as

$$D_\ub f(\xb) = \lim_{h \rightarrow 0}  \frac{f(\xb + h \ub) - f(\xb)}{h},$$

which can be rewritten according to the chain rule as

$$D_\ub f(\xb) = \nabla f(\xb) \cdot \ub.$$

Since $D_\ub f(\xb)$ gives the rates of change of $f$ at the point $\xb$ in all possible directions, to minimize $f$, we are interested in finding the direction where $f$ can be reduced fastest. Thus, we can minimize the directional derivative $D_\ub f(\xb)$ with respect to $\ub$. Since $D_\ub f(\xb) = \|\nabla f(\xb)\| \cdot \|\ub\|  \cdot \text{cos} (\theta) = \|\nabla f(\xb)\|  \cdot \text{cos} (\theta)$, where $\theta$ is the angle between $\nabla f(\xb)$ and $\ub$. The minimum value of $\text{cos}(\theta)$ is -1 when $\theta = \pi$. Therefore, $D_\ub f(\xb)$ is minimized when $\ub$ is at the opposite direction of the gradient $\nabla f(\xb)$. Now we can iteratively reduce the value of $f$ with the following gradient descent update:

$$\xb := \xb - \eta \nabla f(\xb),$$

where the positive scalar $\eta$ is called the learning rate or step size.

## Stochastic gradient descent$\newcommand{\cB}{\mathcal{B}}$

When training deep learning models, we often consider the objective function as a sum of a finite number of functions:

$$f(\xb) = \frac{1}{n} \sum_{i = 1}^n f_i(\xb),$$

where $f_i(\xb)$ is a loss function only based on the training data instance indexed by $i$. It is important to highlight that the per-iteration computational cost in gradient descent scales linearly with the training data set size $n$. Hence, when $n$ is large, the per-iteration computational cost of gradient descent is high.

In view of this, stochastic gradient descent offers a lighter-weight solution. At each iteration, rather than computing the gradient $\nabla f(\xb)$, stochastic gradient descent randomly samples $i$ and computes $\nabla f_i(\xb)$ intead. The insight is, stochastic gradient descent uses $\nabla f_i(\xb)$ to approximate the true gradient $\nabla f(\xb)$. In a generalized case, a mini-batch $\cB$ that consist of data instance indices may be sampled at each iteration to approximate the true gradient to update:

$$\xb := \xb - \eta \frac{1}{|\cB|} \sum_{i \in \cB}\nabla f_i(\xb),$$

where $|\cB|$ denotes the cardinality of the mini-batch.


## Experiments

For demonstrating the aforementioned gradient-based optimization algorithms, we use the regression problem in the [linear regression chapter](./P02-C01-linear-regression-scratch.ipynb) as a case study.

First, We import related libraries, generate the synthetic data, and construct the model.

In [1]:
from __future__ import print_function
import mxnet as mx
from mxnet import autograd
from mxnet import gluon
import numpy as np

X = np.random.randn(10000, 2)
Y = 2 * X[:,0] - 3.4 * X[:,1] + 4.2 + .01 * np.random.normal(size=10000)

ctx = mx.cpu()
net = gluon.nn.Sequential()
net.add(gluon.nn.Dense(1))
net.collect_params().initialize()
loss = gluon.loss.L2Loss()

Then we specify the batch sizes and learning rates for stochastic gradient descent algorithms.
Since the number of samples is 10,000, when the batch size is 10,000, the algorithm is essentially gradient descent.

In [2]:
batch_sizes = [1, 10, 100, 1000, 10000]
learning_rates = [0.1, 0.1, 0.5, 0.5, 1.0]
epochs = 3

Now we are ready to train the models and observe the inferred parameter values after the model training.

In [3]:
for batch_size, learning_rate in zip(batch_sizes, learning_rates):
    trainer = gluon.Trainer(net.collect_params(), 'sgd',
                            {'learning_rate': learning_rate})
    net.collect_params().initialize(mx.init.Xavier(magnitude=2.24),
                                    ctx=ctx, force_reinit=True)
    train_data = mx.io.NDArrayIter(X, Y, batch_size=batch_size, shuffle=True)

    for e in range(epochs):
        train_data.reset()
        for i, batch in enumerate(train_data):
            data = batch.data[0].as_in_context(ctx)
            label = batch.label[0].as_in_context(ctx).reshape((-1, 1))
            with autograd.record():
                output = net(data)
                mse = loss(output, label)
            mse.backward()
            trainer.step(data.shape[0])

    for para_name, para_value in net.collect_params().items():
        print("Batch size:", batch_size, para_name,
              para_value.data().asnumpy()[0])

Batch size: 1 dense0_weight [ 1.99995232 -3.39916563]
Batch size: 1 dense0_bias 4.20145
Batch size: 10 dense0_weight [ 1.99982965 -3.40126681]
Batch size: 10 dense0_bias 4.19938
Batch size: 100 dense0_weight [ 1.99982131 -3.39967561]
Batch size: 100 dense0_bias 4.19952
Batch size: 1000 dense0_weight [ 2.00008678 -3.39985538]
Batch size: 1000 dense0_bias 4.19995
Batch size: 10000 dense0_weight [ 1.9999845  -3.40007067]
Batch size: 10000 dense0_bias 4.19982


As expected, all the investigated algorithms find the weight vector to be close to [2, -3.4] and the bias term to be close to 4.2 as specified when generating the synthetic data.